In [1]:
#Export
import os,uuid,sys,base64,math
import pandas as pd
import numpy as np
import GEOparse
import zipfile
import re

from matplotlib.figure import Figure
from Bio import Entrez

import sklearn
import pandas as pd
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import r,pandas2ri
from rpy2.robjects.conversion import localconverter

In [3]:
ls = ['GSE32280']#'GSE188309','GSE65682','GSE42830','GSE42826',
for i in ls:
    while True:
        try:
            gse = GEOparse.get_GEO(geo=i, destdir="./", silent=False,include_data = True,annotate_gpl=True)
            break
        except OSError:
            continue
    expr = gse.pivot_and_annotate(values='VALUE', gpl=gse.gpls[list(gse.gpls.keys())[0]].table,annotation_column='Gene Symbol',gpl_on='ID',gsm_on='ID_REF').set_index('Gene Symbol').T
    expr = expr.merge(right=gse.phenotype_data,left_index=True,right_index=True)
    # clinical = gse.phenotype_data
    # df = pd.concat([expr,clinical],axis = 1)
    expr.T.to_csv(path_or_buf=f"./{i}_.csv")

10-10月-2023 15:10:32 DEBUG utils - Directory ./ already exists. Skipping.
10-10月-2023 15:10:32 INFO GEOparse - Downloading ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE32nnn/GSE32280/soft/GSE32280_family.soft.gz to ./GSE32280_family.soft.gz
100%|█████████████████████████████████████████████████████████████████████████████| 25.2M/25.2M [00:21<00:00, 1.25MB/s]
10-10月-2023 15:11:37 DEBUG downloader - Size validation passed
10-10月-2023 15:11:37 DEBUG downloader - Moving C:\Users\ADMINI~1\AppData\Local\Temp\tmprqs3l5va to E:\马驰宇\公司项目\DSFJQ230830\01.差异分析\GSE32280_family.soft.gz
10-10月-2023 15:11:37 DEBUG downloader - Successfully downloaded ftp://ftp.ncbi.nlm.nih.gov/geo/series/GSE32nnn/GSE32280/soft/GSE32280_family.soft.gz
10-10月-2023 15:11:37 INFO GEOparse - Parsing ./GSE32280_family.soft.gz: 
10-10月-2023 15:11:37 DEBUG GEOparse - DATABASE: GeoMiame
10-10月-2023 15:11:37 DEBUG GEOparse - SERIES: GSE32280
10-10月-2023 15:11:37 DEBUG GEOparse - PLATFORM: GPL570
D:\anaconda3\lib\site-packages\GEOpa

In [4]:
gse.gpls[list(gse.gpls.keys())[0]].table

,ID,GB_ACC,SPOT_ID,Species Scientific Name,Annotation Date,Sequence Type,Sequence Source,Target Description,Representative Public ID,Gene Title,Gene Symbol,ENTREZ_GENE_ID,RefSeq Transcript ID,Gene Ontology Biological Process,Gene Ontology Cellular Component,Gene Ontology Molecular Function
0,1007_s_at,U48705,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,Affymetrix Proprietary Database,U48705 /FEATURE=mRNA /DEFINITION=HSU48705 Huma...,U48705,discoidin domain receptor tyrosine kinase 1 //...,DDR1 /// MIR4640,780 /// 100616237,NM_001202521 /// NM_001202522 /// NM_001202523...,0001558 // regulation of cell growth // inferr...,0005576 // extracellular region // inferred fr...,0000166 // nucleotide binding // inferred from...
1,1053_at,M87338,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,GenBank,M87338 /FEATURE= /DEFINITION=HUMA1SBU Human re...,M87338,"replication factor C (activator 1) 2, 40kDa",RFC2,5982,NM_001278791 /// NM_001278792 /// NM_001278793...,0000278 // mitotic cell cycle // traceable aut...,0005634 // nucleus // inferred from electronic...,0000166 // nucleotide binding // inferred from...
2,117_at,X51757,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,Affymetrix Proprietary Database,X51757 /FEATURE=cds /DEFINITION=HSP70B Human h...,X51757,heat shock 70kDa protein 6 (HSP70B'),HSPA6,3310,NM_002155,0000902 // cell morphogenesis // inferred from...,0005737 // cytoplasm // inferred from direct a...,0000166 // nucleotide binding // inferred from...
3,121_at,X69699,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,GenBank,X69699 /FEATURE= /DEFINITION=HSPAX8A H.sapiens...,X69699,paired box 8,PAX8,7849,NM_003466 /// NM_013951 /// NM_013952 /// NM_0...,0001655 // urogenital system development // in...,0005634 // nucleus // inferred from direct ass...,0000979 // RNA polymerase II core promoter seq...
4,1255_g_at,L36861,NaN,Homo sapiens,"Oct 6, 2014",Exemplar sequence,Affymetrix Proprietary Database,L36861 /FEATURE=expanded_cds /DEFINITION=HUMGC...,L36861,guanylate cyclase activator 1A (retina),GUCA1A,2978,NM_000409 /// XM_006715073,0007165 // signal transduction // non-traceabl...,0001750 // photoreceptor outer segment // infe...,0005509 // calcium ion binding // inferred fro...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54670,AFFX-ThrX-5_at,NaN,--Control,Homo sapiens,"Oct 6, 2014",Control sequence,Affymetrix Proprietary Database,B. subtilis /GEN=thrC /DB_XREF=gb:X04603.1 /NO...,AFFX-ThrX-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54671,AFFX-ThrX-M_at,NaN,--Control,Homo sapiens,"Oct 6, 2014",Control sequence,Affymetrix Proprietary Database,"B. subtilis /GEN=thrC, thrB /DB_XREF=gb:X04603...",AFFX-ThrX-M,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54672,AFFX-TrpnX-3_at,NaN,--Control,Homo sapiens,"Oct 6, 2014",Control sequence,Affymetrix Proprietary Database,"B. subtilis /GEN=trpC, trpF /DB_XREF=gb:K01391...",AFFX-TrpnX-3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
54673,AFFX-TrpnX-5_at,NaN,--Control,Homo sapiens,"Oct 6, 2014",Control sequence,Affymetrix Proprietary Database,"B. subtilis /GEN=trpE, trpD /DB_XREF=gb:K01391...",AFFX-TrpnX-5,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [2]:
pandas2ri.activate()
ro.r('''library(limma)''')

array(['limma', 'tools', 'stats', 'graphics', 'grDevices', 'utils',
       'datasets', 'methods', 'base'], dtype='<U9')

In [3]:
def gene_data_preprocess(df_raw, predclass, test, con, batch=0, imputeNA=0, filt_outlier=1):
    ## 检查特征名称是否合规，包括空值、同义名、重名
    # df_raw=df_raw.T
    # df_raw.iloc[:,:-1][df_raw.iloc[:,:-1].astype('float')<0] = 0.000000001
    # df_raw=df_raw.T
    df_raw=df_raw[~pd.isna(df_raw.index)]
    df_raw=df_raw[df_raw.index.str.contains('/')==False]
    ro.globalenv['rdf_raw']=pandas2ri.py2rpy(df_raw.drop([predclass,batch]).astype('float'))
    ro.globalenv['genes']=pandas2ri.py2rpy(df_raw.drop([predclass,batch]).index)
    rdf_raw=ro.r('''df_raw = as.data.frame(avereps(rdf_raw,ID=genes))''')
    df_raw=pd.concat([rdf_raw,pd.DataFrame(df_raw.loc[[predclass,batch],:])])
    # df_raw=pd.concat([df_raw.drop(predclass).groupby(level=0).mean(),pd.DataFrame(df_raw.loc[predclass,:]).T])
    df_raw=df_raw.T        
    ## 按照每个观察值缺失或为0的特征数，按百分位法去离群值
    if filt_outlier == 1:
        # within 2 standerd deviation
        # df_dat[np.abs(df_dat.A - df_dat.A.mean()) <= (2 * df_dat.A.std())]
        countsNa=~((df_raw.drop([predclass,batch],axis=1)==0) ==\
               (pd.isnull(df_raw.drop([predclass,batch],axis=1)))).sum(axis=1)
        iqr=np.percentile(countsNa,75,axis=0)-np.percentile(countsNa,25,axis=0)
        lowerlimt,upperlimt = np.percentile(countsNa,25,axis=0)-1.5*iqr,np.percentile(countsNa,75,axis=0)+1.5*iqr
        df_raw=df_raw[[x<=upperlimt for x in countsNa]]
    ## KNN法补值
    if imputeNA == 1:
        imp_knn = KNNImputer(n_neighbors=5)
        df_raw = pd.concat([pd.DataFrame(imp_knn.fit_transform(df_raw.drop([predclass,batch],axis=1)),\
                  columns = df_raw.columns.drop([predclass,batch]),index=df_raw.index),df_raw[predclass,batch]],axis=1)
    ## 检查数据是否已log2转换
    qx = np.percentile(df_raw.iloc[:,:-2],[0,25,50,75,99,100])
    LogC=((qx[4]>100)|(((qx[5]-qx[0])>50) & (qx[1]>0)))
    if LogC:
        df1 = df_raw.drop(predclass,axis=1).astype('float')
        df1[df1<0] = 0.000000001
        df1[predclass] = df_raw[predclass]
        df_raw = df1
        df_raw[df_raw.columns.drop([predclass,batch])]=np.log2(df_raw.drop([predclass,batch],axis=1).astype('float')+1).values 
    # if batch != 0:
    #     df_raw.iloc[:,:-2]=pd.DataFrame(ro.r.normalizeBetweenArrays\
    #                     (df_raw.drop([predclass,batch],axis=1).astype('float').T)).T.values
    # else:
    #     df_raw.iloc[:,:-1]=pd.DataFrame(ro.r.normalizeBetweenArrays\
    #                             (df_raw.drop([predclass],axis=1).astype('float').T)).T.values
    df_raw[predclass] = [i.replace(' ','_') for i in df_raw[predclass]]
    return df_raw

In [6]:
df_raw = pd.read_csv("E:/马驰宇/公司项目/已完成/DSHT230519/DSHT230519_rna_wes_hcc_biomarker/GSE230400.csv",index_col=0)

In [149]:
df_raw =df_raw.loc[:,df_raw.iloc[-1,:]=='no']

In [150]:
df_raw.rename(index={df_raw.index[-3]:'group',df_raw.index[-2]:'batch'},inplace=True)

In [151]:
df_raw = df_raw.iloc[:-1,:]

In [7]:
df_raw = gene_data_preprocess(df_raw,'group',None,'Sham','batch')

D:\anaconda3\lib\site-packages\rpy2\robjects\pandas2ri.py:56: UserWarning: DataFrame contains duplicated elements in the index, which will lead to loss of the row names in the resulting data.frame
  warnings.warn('DataFrame contains duplicated elements in the index, '


In [8]:
con='Sham'
predclass = 'group'
batch = 'batch'

In [19]:
df_raw=df_raw.T

In [20]:
df_raw=pd.concat([df_raw.drop(predclass,axis=1).astype('float'),pd.DataFrame(df_raw.loc[:,predclass])],axis=1)

In [21]:
df_raw.T.to_csv('./ori_GSE230400.csv')

In [185]:

disease = list(set(df_raw[predclass]))
disease.remove(con)
disease = disease[0]
## limma差异分析
ro.globalenv['rpred'] = predclass
ro.globalenv['rcon'] = con.replace(' ', '_')
ro.globalenv['rdisease'] = disease.replace(' ', '_')
ro.globalenv['rdf_raw'] = pandas2ri.py2rpy(df_raw)
ro.r('''
design <- as.data.frame(model.matrix(~0+factor(rdf_raw[,rpred])))
rownames(design) <- rownames(rdf_raw)
colnames(design) <- unique(rdf_raw[,rpred])
data <- as.data.frame(t(rdf_raw[,-match(c(rpred),colnames(rdf_raw))]))
fit <- lmFit(data,design)
cont.matrix <- makeContrasts(paste0(rdisease,'-',rcon),levels=design)
fit2 <- contrasts.fit(fit,cont.matrix)
fit2 <- eBayes(fit2)
genestat <- topTable(fit2,adjust = 'fdr',number=200000)
''')
genestat = ro.r('genestat')
genestat.to_csv(path_or_buf='./genestat_GSE32280.csv')

In [22]:
disease = list(set(df_raw[predclass]))
disease.remove(con)
disease = disease[0]
## limma差异分析
ro.globalenv['rpred'] = predclass
ro.globalenv['rbatch'] = batch
ro.globalenv['rcon'] = con.replace(' ', '_')
ro.globalenv['rdisease'] = disease.replace(' ', '_')
ro.globalenv['rdf_raw'] = pandas2ri.py2rpy(df_raw)
ro.r('''
design <- as.data.frame(model.matrix(~0+factor(rdf_raw[,rpred])))
rownames(design) <- rownames(rdf_raw)
colnames(design) <- unique(rdf_raw[,rpred])
data <- as.data.frame(t(rdf_raw[,-match(c(rpred,rbatch),colnames(rdf_raw))]))
data <- removeBatchEffect(data,batch=rdf_raw[,rbatch],design=design)
fit <- lmFit(data,design)
cont.matrix <- makeContrasts(paste0(rdisease,'-',rcon),levels=design)
fit2 <- contrasts.fit(fit,cont.matrix)
fit2 <- eBayes(fit2)
data <- cbind(as.data.frame(t(data)),rdf_raw[,c(rpred,rbatch)])
genestat <- topTable(fit2,adjust = 'fdr',number=200000)
''')
genestat = ro.r('genestat')
df_raw_ = ro.r('data')
genestat.to_csv(path_or_buf='./genestat_GSE230400.csv')
df_raw_.T.to_csv(path_or_buf="./batch_GSE230400.csv")

In [23]:
diffgene = list(genestat.index[(genestat['P.Value']<0.05) & (abs(genestat['logFC'])>math.log2(1.2))])
# if len(diffgene)<10:
#     diffgene = list(trainstat.index[(trainstat['P.Value']<0.05) & (abs(trainstat['logFC'])>math.log2(1.2))]\
#                 .intersection(teststat.index[(teststat['P.Value']<1) &(abs(teststat['logFC'])>=math.log2(1))]))
# diffgene = list(trainstat.index[(trainstat['adj.P.Val']<0.001) & (abs(trainstat['logFC'])>math.log2(2))])
df_raw_[diffgene+[predclass]].T.to_csv(path_or_buf='./df_diff_GSE230400.csv')

In [78]:
list(set(df_raw[predclass]))

['MDD_patient', 'healthy_control']

In [80]:
con

'healthy control'

In [119]:
ro.r('removeBatchEffect(data,batch=c(rdf_raw[,rbatch][1:12],rdf_raw[,rbatch][1:12]+1))')

R[write to console]: Error in getEAWP(object) : 
  Data object doesn't contain numeric expression values



RRuntimeError: Error in getEAWP(object) : 
  Data object doesn't contain numeric expression values


In [120]:
ro.r('c(rdf_raw[,rbatch][1:12],rdf_raw[,rbatch][1:12]+1)')

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 2., 2., 2., 2., 2.,
       2., 2., 2., 2., 2., 2., 2.])

In [122]:
ro.r('data')

,RFC2,HSPA6,PAX8,GUCA1A,THRA,PTPN21,CCL5,CYP2E1,EPHB3,ESRRA,...,CCDC63,RP11-495P10.6,LOC101929475,RP11-1260E13.2,LOC389641,GALR3,NUS1P3,LOC100505915,group,batch
GSM799722,6.148388,8.045257,4.232761,3.839566,4.183832,2.916423,11.227896,3.198198,4.217429,6.238428,...,3.253855,3.249355,3.161239,2.421982,4.933923,5.794943,4.129727,5.524346,MDD_patient,1.0
GSM799723,6.142244,8.465042,4.222489,3.600137,4.179460,2.931348,11.146679,3.194737,4.274186,6.327059,...,3.268230,3.414947,3.259012,2.381827,5.020641,5.835959,4.177971,5.384054,MDD_patient,1.0
GSM799724,5.883527,7.905676,4.429704,3.665847,4.208306,3.154402,11.346577,3.120005,4.323584,5.780105,...,3.601231,3.664376,3.454774,2.734008,4.836319,5.938310,3.804713,5.094640,MDD_patient,1.0
GSM799725,6.095919,8.370535,4.387908,3.616114,4.287185,2.996327,11.513954,3.305829,4.262662,6.339603,...,3.450441,3.528272,3.316020,2.344654,4.317263,5.987785,4.330108,5.302916,MDD_patient,1.0
GSM799726,6.090770,7.924438,4.331669,3.681631,4.261576,2.904393,11.028056,3.359196,4.512784,6.090708,...,3.601231,3.518069,3.304104,2.262178,4.412006,5.814983,4.258313,5.693921,MDD_patient,1.0
GSM799727,6.222332,8.716242,4.340671,3.724592,4.391088,2.923516,10.503290,3.228336,4.433423,6.459870,...,3.499685,3.610751,3.304104,2.469316,4.611620,5.815193,3.955750,5.062008,healthy_control,1.0
GSM799728,6.258152,7.949693,4.173304,3.438605,4.387028,2.963415,11.459007,3.222219,4.149369,6.030153,...,3.263615,3.553551,3.316020,2.390092,4.524156,5.666899,4.785057,5.633885,healthy_control,1.0
GSM799729,5.900160,7.423085,4.291238,3.706790,4.298599,3.017497,11.235926,3.161197,4.421870,5.937494,...,3.939465,3.520045,3.581988,2.602588,4.432507,5.798980,4.001823,5.513076,healthy_control,1.0
GSM799730,5.877479,8.232169,4.295591,3.365712,4.294577,3.044343,11.213387,3.294066,4.492464,6.234322,...,3.680076,3.433126,3.280869,2.344654,4.685203,5.614754,3.993753,5.500660,healthy_control,1.0
GSM799731,6.439968,6.933083,4.309157,3.586180,4.428256,2.888381,10.349664,3.403184,4.233075,6.364375,...,3.352993,3.536246,3.307099,2.576507,4.530699,5.593468,5.078801,5.564357,healthy_control,1.0


,RFC2,HSPA6,PAX8,GUCA1A,THRA,PTPN21,CCL5,CYP2E1,EPHB3,ESRRA,...,CCDC63,RP11-495P10.6,LOC101929475,RP11-1260E13.2,LOC389641,GALR3,NUS1P3,LOC100505915,batch,group
GSM799722,6.148388,8.045257,4.232761,3.839566,4.183832,2.916423,11.227896,3.198198,4.217429,6.238428,...,3.253855,3.249355,3.161239,2.421982,4.933923,5.794943,4.129727,5.524346,1.0,MDD_patient
GSM799723,6.142244,8.465042,4.222489,3.600137,4.179460,2.931348,11.146679,3.194737,4.274186,6.327059,...,3.268230,3.414947,3.259012,2.381827,5.020641,5.835959,4.177971,5.384054,1.0,MDD_patient
GSM799724,5.883527,7.905676,4.429704,3.665847,4.208306,3.154402,11.346577,3.120005,4.323584,5.780105,...,3.601231,3.664376,3.454774,2.734008,4.836319,5.938310,3.804713,5.094640,1.0,MDD_patient
GSM799725,6.095919,8.370535,4.387908,3.616114,4.287185,2.996327,11.513954,3.305829,4.262662,6.339603,...,3.450441,3.528272,3.316020,2.344654,4.317263,5.987785,4.330108,5.302916,1.0,MDD_patient
GSM799726,6.090770,7.924438,4.331669,3.681631,4.261576,2.904393,11.028056,3.359196,4.512784,6.090708,...,3.601231,3.518069,3.304104,2.262178,4.412006,5.814983,4.258313,5.693921,1.0,MDD_patient
GSM799727,6.222332,8.716242,4.340671,3.724592,4.391088,2.923516,10.503290,3.228336,4.433423,6.459870,...,3.499685,3.610751,3.304104,2.469316,4.611620,5.815193,3.955750,5.062008,1.0,healthy_control
GSM799728,6.258152,7.949693,4.173304,3.438605,4.387028,2.963415,11.459007,3.222219,4.149369,6.030153,...,3.263615,3.553551,3.316020,2.390092,4.524156,5.666899,4.785057,5.633885,1.0,healthy_control
GSM799729,5.900160,7.423085,4.291238,3.706790,4.298599,3.017497,11.235926,3.161197,4.421870,5.937494,...,3.939465,3.520045,3.581988,2.602588,4.432507,5.798980,4.001823,5.513076,1.0,healthy_control
GSM799730,5.877479,8.232169,4.295591,3.365712,4.294577,3.044343,11.213387,3.294066,4.492464,6.234322,...,3.680076,3.433126,3.280869,2.344654,4.685203,5.614754,3.993753,5.500660,1.0,healthy_control
GSM799731,6.439968,6.933083,4.309157,3.586180,4.428256,2.888381,10.349664,3.403184,4.233075,6.364375,...,3.352993,3.536246,3.307099,2.576507,4.530699,5.593468,5.078801,5.564357,1.0,healthy_control
